In [4]:
# Load necessary libraries
library(SingleCellExperiment)
library(scmap)
library(zellkonverter)  # For reading h5ad files
library(dplyr)
library(Matrix)
library(ggplot2)
library(tibble)
library(yaml)

# Set parameters
data_path <- "/home/project/11003054/changxu/Projects/DIRAC/Section-4"
methods <- "SCMAP"
sim_methods <- "enhanced"
omics <- "RNA"

# Define color palette
colormaps <- c("#E41A1C", "#377EB8", "#4DAF4A", "#984EA3", "#FF7F00", "#A65628", "#FFFF33", "#F781BF", "#999999",
               "#E5D8BD", "#B3CDE3", "#CCEBC5", "#FED9A6", "#FBB4AE", "#8DD3C7", "#BEBADA", "#80B1D3", "#B3DE69", "#FCCDE5",
               "#BC80BD", "#FFED6F", "#8DA0CB", "#E78AC3", "#E5C494", "#CCCCCC", "#FB9A99", "#E31A1C", "#CAB2D6", "#6A3D9A", 
               "#B15928")

# Create a list to save all results
results_list <- list()

for (i in 0:3) {
  cat(sprintf("Processing dataset i = %d...\n", i))
  
  # 1. Load the data
  file_path <- sprintf("%s/scMultiSim_data/sim-%s/sim_%s_%d.h5ad", data_path, sim_methods, omics, i)
  sce_all <- readH5AD(file_path)
  
  # 2. Preprocessing - Fix the assay access
  # First check what assays are available
  cat("Available assays:", names(assays(sce_all)), "\n")
  
  # Typically the main matrix is in "X" for h5ad files
  # Create logcounts from the raw counts
  if (!"counts" %in% names(assays(sce_all))) {
    # Try to use "X" if "counts" doesn't exist
    if ("X" %in% names(assays(sce_all))) {
      assay(sce_all, "counts") <- assay(sce_all, "X")
    } else {
      stop("No counts or X assay found in the SCE object")
    }
  }
  
  # Now create logcounts
  logcounts(sce_all) <- log2(assay(sce_all, "counts") + 1)
  rowData(sce_all)$feature_symbol <- rownames(sce_all)
  sce_all <- sce_all[!duplicated(rownames(sce_all)), ]
  
  # Inner loop: j = 1 to 5
  for (j in 1:5) {
    cat(sprintf("  Current batch = %d\n", j))
    
    # 3. Split into source and target
    source_sce <- sce_all[, sce_all$batch != j]
    target_sce <- sce_all[, sce_all$batch == j]
    
    # 4. Feature selection (only on the source data)
    source_sce <- selectFeatures(source_sce, n_features = 100, suppress_plot = TRUE) 
    # rowData(source_sce)$scmap_features <- TRUE
    
    # 5. Build index
    source_sce <- indexCluster(source_sce, cluster_col = "cell.type")
    
    # 6. Predict target
    scmapCluster_results <- scmapCluster(
      projection = target_sce,
      index_list = list(source = metadata(source_sce)$scmap_cluster_index)
    )
    
    # 7. Evaluate results
    pred_labels <- as.character(scmapCluster_results$combined_labs)
    true_labels <- as.character(colData(target_sce)$cell.type)

    # Exclude unassigned predictions
    valid_idx <- pred_labels != "unassigned"
    unassigned_count <- sum(pred_labels == "unassigned")
    cat(sprintf("Number of 'unassigned' predictions: %d\n", unassigned_count))

    if (sum(valid_idx) > 0) {
      cm <- caret::confusionMatrix(
        factor(pred_labels[valid_idx], levels = unique(true_labels)),
        factor(true_labels[valid_idx], levels = unique(true_labels))
      )

      accuracy <- replace(cm$overall["Accuracy"], is.na(cm$overall["Accuracy"]), 0)
      precision <- mean(replace(cm$byClass[,"Precision"], is.na(cm$byClass[,"Precision"]), 0))
      recall <- mean(replace(cm$byClass[,"Recall"], is.na(cm$byClass[,"Recall"]), 0))
      f1 <- mean(replace(cm$byClass[,"F1"], is.na(cm$byClass[,"F1"]), 0))

      metrics_all <- list(
        "Accuracy Score" = accuracy,
        "Precision Score" = precision,
        "Recall Score" = recall,
        "F1 Score" = f1
      )}
      
    print(metrics_all)
    
    # 8. Save results
    save_path <- sprintf("%s/Results/merged_%s_%s_%s_%d_%d", data_path, omics, methods, sim_methods, i, j)
    if (!dir.exists(save_path)) {
      dir.create(save_path, recursive = TRUE)
    }
    
    # Save evaluation metrics
    write_yaml(metrics_all, file.path(save_path, "annotate_metrics.yaml"))
    
    # Save predictions
    output_df <- data.frame(
      Ground_Truth = true_labels,
      Predicted = pred_labels,
      stringsAsFactors = FALSE
    )
    write.csv(output_df, file.path(save_path, sprintf("%s_%s_%s.csv", omics, methods, sim_methods)), row.names = FALSE)
    
    # Plot results (optional)
    plot_df <- output_df %>% count(Ground_Truth, Predicted)
    
    p <- ggplot(plot_df, aes(x = Ground_Truth, y = n, fill = Predicted)) +
      geom_bar(stat = "identity", position = "dodge") +
      theme_minimal() +
      scale_fill_manual(values = colormaps) +
      labs(title = sprintf("i=%d, j=%d: scmap Results", i, j),
           x = "True Labels", y = "Count", fill = "Predicted Labels")
    
    ggsave(file.path(save_path, sprintf("%s_%s_%s.pdf", omics, methods, sim_methods)), plot = p, width = 8, height = 6)
    
    # Collect results
    results_list[[paste0("i", i, "_j", j)]] <- metrics_all
  }
}

# 9. Summarize all results
summary_df <- bind_rows(results_list, .id = "dataset")
write.csv(summary_df, file.path(data_path, "Results", sprintf("sim_metrics_%s_%s_%s.csv", omics, methods, sim_methods)), row.names = FALSE)

cat("All processing completed!\n")

Processing dataset i = 0...
Available assays: X 
  Current batch = 1
Number of 'unassigned' predictions: 51
$`Accuracy Score`
 Accuracy 
0.8224613 

$`Precision Score`
[1] 0.8168036

$`Recall Score`
[1] 0.845764

$`F1 Score`
[1] 0.8295106

  Current batch = 2
Number of 'unassigned' predictions: 48
$`Accuracy Score`
 Accuracy 
0.8148407 

$`Precision Score`
[1] 0.815627

$`Recall Score`
[1] 0.8452946

$`F1 Score`
[1] 0.8274504

  Current batch = 3
Number of 'unassigned' predictions: 53
$`Accuracy Score`
 Accuracy 
0.8223126 

$`Precision Score`
[1] 0.813125

$`Recall Score`
[1] 0.8438593

$`F1 Score`
[1] 0.8263964

  Current batch = 4
Number of 'unassigned' predictions: 51
$`Accuracy Score`
Accuracy 
0.809268 

$`Precision Score`
[1] 0.7991392

$`Recall Score`
[1] 0.8308642

$`F1 Score`
[1] 0.8115352

  Current batch = 5
Number of 'unassigned' predictions: 54
$`Accuracy Score`
 Accuracy 
0.8224456 

$`Precision Score`
[1] 0.8183882

$`Recall Score`
[1] 0.8478307

$`F1 Score`
[1] 0.83009

In [5]:
# Load necessary libraries
library(SingleCellExperiment)
library(scmap)
library(zellkonverter)  # For reading h5ad files
library(dplyr)
library(Matrix)
library(ggplot2)
library(tibble)
library(yaml)

# Set parameters
data_path <- "/home/project/11003054/changxu/Projects/DIRAC/Section-4"
methods <- "SCMAP"
sim_methods <- "layers"
omics <- "RNA"

# Define color palette
colormaps <- c("#E41A1C", "#377EB8", "#4DAF4A", "#984EA3", "#FF7F00", "#A65628", "#FFFF33", "#F781BF", "#999999",
               "#E5D8BD", "#B3CDE3", "#CCEBC5", "#FED9A6", "#FBB4AE", "#8DD3C7", "#BEBADA", "#80B1D3", "#B3DE69", "#FCCDE5",
               "#BC80BD", "#FFED6F", "#8DA0CB", "#E78AC3", "#E5C494", "#CCCCCC", "#FB9A99", "#E31A1C", "#CAB2D6", "#6A3D9A", 
               "#B15928")

# Create a list to save all results
results_list <- list()

# Outer loop: i = 0 to 7
for (i in 0:3) {
  cat(sprintf("Processing dataset i = %d...\n", i))
  
  # 1. Load the data
  file_path <- sprintf("%s/scMultiSim_data/sim-%s/sim_%s_%d.h5ad", data_path, sim_methods, omics, i)
  sce_all <- readH5AD(file_path)
  
  # 2. Preprocessing - Fix the assay access
  # First check what assays are available
  cat("Available assays:", names(assays(sce_all)), "\n")
  
  # Typically the main matrix is in "X" for h5ad files
  # Create logcounts from the raw counts
  if (!"counts" %in% names(assays(sce_all))) {
    # Try to use "X" if "counts" doesn't exist
    if ("X" %in% names(assays(sce_all))) {
      assay(sce_all, "counts") <- assay(sce_all, "X")
    } else {
      stop("No counts or X assay found in the SCE object")
    }
  }
  
  # Now create logcounts
  logcounts(sce_all) <- log2(assay(sce_all, "counts") + 1)
  rowData(sce_all)$feature_symbol <- rownames(sce_all)
  sce_all <- sce_all[!duplicated(rownames(sce_all)), ]
  
  # Inner loop: j = 1 to 5
  for (j in 1:5) {
    cat(sprintf("  Current batch = %d\n", j))
    
    # 3. Split into source and target
    source_sce <- sce_all[, sce_all$batch != j]
    target_sce <- sce_all[, sce_all$batch == j]
    
    # 4. Feature selection (only on the source data)
    # rowData(source_sce)$scmap_features <- TRUE
    source_sce <- selectFeatures(source_sce, n_features = 100, suppress_plot = TRUE)
    
    # 5. Build index
    source_sce <- indexCluster(source_sce, cluster_col = "cell.type")
    
    # 6. Predict target
    scmapCluster_results <- scmapCluster(
      projection = target_sce,
      index_list = list(source = metadata(source_sce)$scmap_cluster_index)
    )
    
    # 7. Evaluate results
    pred_labels <- as.character(scmapCluster_results$combined_labs)
    true_labels <- as.character(colData(target_sce)$cell.type)

    # Exclude unassigned predictions
    valid_idx <- pred_labels != "unassigned"
    unassigned_count <- sum(pred_labels == "unassigned")
    cat(sprintf("Number of 'unassigned' predictions: %d\n", unassigned_count))

    if (sum(valid_idx) > 0) {
      cm <- caret::confusionMatrix(
        factor(pred_labels[valid_idx], levels = unique(true_labels)),
        factor(true_labels[valid_idx], levels = unique(true_labels))
      )

      accuracy <- replace(cm$overall["Accuracy"], is.na(cm$overall["Accuracy"]), 0)
      precision <- mean(replace(cm$byClass[,"Precision"], is.na(cm$byClass[,"Precision"]), 0))
      recall <- mean(replace(cm$byClass[,"Recall"], is.na(cm$byClass[,"Recall"]), 0))
      f1 <- mean(replace(cm$byClass[,"F1"], is.na(cm$byClass[,"F1"]), 0))

      metrics_all <- list(
        "Accuracy Score" = accuracy,
        "Precision Score" = precision,
        "Recall Score" = recall,
        "F1 Score" = f1
      )}

    print(metrics_all)
    
    # 8. Save results
    save_path <- sprintf("%s/Results/merged_%s_%s_%s_%d_%d", data_path, omics, methods, sim_methods, i, j)
    if (!dir.exists(save_path)) {
      dir.create(save_path, recursive = TRUE)
    }
    
    # Save evaluation metrics
    write_yaml(metrics_all, file.path(save_path, "annotate_metrics.yaml"))
    
    # Save predictions
    output_df <- data.frame(
      Ground_Truth = true_labels,
      Predicted = pred_labels,
      stringsAsFactors = FALSE
    )
    write.csv(output_df, file.path(save_path, sprintf("%s_%s_%s.csv", omics, methods, sim_methods)), row.names = FALSE)
    
    # Plot results (optional)
    plot_df <- output_df %>% count(Ground_Truth, Predicted)
    
    p <- ggplot(plot_df, aes(x = Ground_Truth, y = n, fill = Predicted)) +
      geom_bar(stat = "identity", position = "dodge") +
      theme_minimal() +
      scale_fill_manual(values = colormaps) +
      labs(title = sprintf("i=%d, j=%d: scmap Results", i, j),
           x = "True Labels", y = "Count", fill = "Predicted Labels")
    
    ggsave(file.path(save_path, sprintf("%s_%s_%s.pdf", omics, methods, sim_methods)), plot = p, width = 8, height = 6)
    
    # Collect results
    results_list[[paste0("i", i, "_j", j)]] <- metrics_all
  }
}

# 9. Summarize all results
summary_df <- bind_rows(results_list, .id = "dataset")
write.csv(summary_df, file.path(data_path, "Results", sprintf("sim_metrics_%s_%s_%s.csv", omics, methods, sim_methods)), row.names = FALSE)

cat("All processing completed!\n")

Processing dataset i = 0...
Available assays: X 
  Current batch = 1
Number of 'unassigned' predictions: 48
$`Accuracy Score`
 Accuracy 
0.8172657 

$`Precision Score`
[1] 0.8128666

$`Recall Score`
[1] 0.8346995

$`F1 Score`
[1] 0.8212469

  Current batch = 2
Number of 'unassigned' predictions: 36
$`Accuracy Score`
 Accuracy 
0.8062043 

$`Precision Score`
[1] 0.8049853

$`Recall Score`
[1] 0.8307671

$`F1 Score`
[1] 0.8150714

  Current batch = 3
Number of 'unassigned' predictions: 41
$`Accuracy Score`
 Accuracy 
0.8240459 

$`Precision Score`
[1] 0.8145859

$`Recall Score`
[1] 0.836402

$`F1 Score`
[1] 0.8232931

  Current batch = 4
Number of 'unassigned' predictions: 39
$`Accuracy Score`
 Accuracy 
0.8143813 

$`Precision Score`
[1] 0.8043719

$`Recall Score`
[1] 0.834722

$`F1 Score`
[1] 0.8160694

  Current batch = 5
Number of 'unassigned' predictions: 37
$`Accuracy Score`
 Accuracy 
0.8337775 

$`Precision Score`
[1] 0.8251176

$`Recall Score`
[1] 0.8535089

$`F1 Score`
[1] 0.83